In [42]:
# Importing Libraries
import random
import numpy as np
import pandas as pd
from scipy import stats
from scipy.signal import find_peaks
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from joblib import load, dump

import warnings
warnings.filterwarnings("ignore")

# **Statistical Measures**

In [43]:
lambda_functions = {
    'mean': lambda x: x.max(),
    'std': lambda x: x.std(),
    'aad': lambda x: np.mean(np.absolute(x - np.mean(x))),
    'min': lambda x: x.min(),
    'max': lambda x: x.max(),
    'meadian': lambda x: np.median(x),
    'mad': lambda x: np.median(np.absolute(x - np.median(x))),
    'iqr': lambda x: np.percentile(x, 75) - np.percentile(x, 25),
    'neg_count': lambda x: np.sum(x < 0),
    'pos_count': lambda x: np.sum(x > 0),
    'above_mean': lambda x: np.sum(x > x.mean()),
    'peak_count': lambda x: len(find_peaks(x)[0]),
    'skewness': lambda x: stats.skew(x),
    'kurtosis': lambda x: stats.kurtosis(x),
    'energy': lambda x: np.sum(x**2)/100,  
}

In [48]:
def preprocess(df):
    ax_list, ay_list, az_list = [], [], []
    gx_list, gy_list, gz_list = [], [], []

    window_size = 50
    step_size = 10

    # creating overlaping windows of size window-size
    ax = df['ax'].values[:window_size]
    ay = df['ay'].values[:window_size]
    az = df['az'].values[:window_size]

    gx = df['gx'].values[:window_size]
    gy = df['gy'].values[:window_size]
    gz = df['gz'].values[:window_size]

    ax_list.append(ax)
    ay_list.append(ay)
    az_list.append(az)

    gx_list.append(gx)
    gy_list.append(gy)
    gz_list.append(gz)

    # converting the signals from time domain to frequency domain using FFT
    ax_list_fft = pd.Series(ax_list).apply(lambda x: np.abs(np.fft.fft(x))[1:step_size+1])
    ay_list_fft = pd.Series(ay_list).apply(lambda x: np.abs(np.fft.fft(x))[1:step_size+1])
    az_list_fft = pd.Series(az_list).apply(lambda x: np.abs(np.fft.fft(x))[1:step_size+1])

    gx_list_fft = pd.Series(gx_list).apply(lambda x: np.abs(np.fft.fft(x))[1:step_size+1])
    gy_list_fft = pd.Series(gy_list).apply(lambda x: np.abs(np.fft.fft(x))[1:step_size+1])
    gz_list_fft = pd.Series(gz_list).apply(lambda x: np.abs(np.fft.fft(x))[1:step_size+1])

    df_new = pd.DataFrame()
    for key, value in lambda_functions.items():
        df_new[f'ax_{key}'] = pd.Series(ax_list).apply(value)
        df_new[f'ay_{key}'] = pd.Series(ay_list).apply(value)
        df_new[f'az_{key}'] = pd.Series(az_list).apply(value)

        df_new[f'gx_{key}'] = pd.Series(gx_list).apply(value)
        df_new[f'gy_{key}'] = pd.Series(gy_list).apply(value)
        df_new[f'gz_{key}'] = pd.Series(gz_list).apply(value)

        df_new[f'ax_{key}_fft'] = pd.Series(ax_list_fft).apply(value)
        df_new[f'ay_{key}_fft'] = pd.Series(ay_list_fft).apply(value)
        df_new[f'az_{key}_fft'] = pd.Series(az_list_fft).apply(value)

        df_new[f'gx_{key}_fft'] = pd.Series(gx_list_fft).apply(value)
        df_new[f'gy_{key}_fft'] = pd.Series(gy_list_fft).apply(value)
        df_new[f'gz_{key}_fft'] = pd.Series(gz_list_fft).apply(value)

    df_new['ax_maxmin_diff'] = df_new['ax_max'] - df_new['ax_min']
    df_new['ay_maxmin_diff'] = df_new['ay_max'] - df_new['ay_min']
    df_new['az_maxmin_diff'] = df_new['az_max'] - df_new['az_min']

    df_new['gx_maxmin_diff'] = df_new['gx_max'] - df_new['gx_min']
    df_new['gy_maxmin_diff'] = df_new['gy_max'] - df_new['gy_min']
    df_new['gz_maxmin_diff'] = df_new['gz_max'] - df_new['gz_min']

    df_new['ax_maxmin_diff_fft'] = df_new['ax_max_fft'] - df_new['ax_min_fft']
    df_new['ay_maxmin_diff_fft'] = df_new['ay_max_fft'] - df_new['ay_min_fft']
    df_new['az_maxmin_diff_fft'] = df_new['az_max_fft'] - df_new['az_min_fft']

    df_new['gx_maxmin_diff_fft'] = df_new['gx_max_fft'] - df_new['gx_min_fft']
    df_new['gy_maxmin_diff_fft'] = df_new['gy_max_fft'] - df_new['gy_min_fft']
    df_new['gz_maxmin_diff_fft'] = df_new['gz_max_fft'] - df_new['gz_min_fft']

    # avg resultant
    df_new['a_avg_result'] = [i.mean() for i in ((pd.Series(ax_list)**2 + pd.Series(ay_list)**2 + pd.Series(az_list)**2)**0.5)]
    df_new['g_avg_result'] = [i.mean() for i in ((pd.Series(gx_list)**2 + pd.Series(gy_list)**2 + pd.Series(gz_list)**2)**0.5)]

    df_new['a_avg_result_fft'] = [i.mean() for i in ((pd.Series(ax_list_fft)**2 + pd.Series(ay_list_fft)**2 + pd.Series(az_list_fft)**2)**0.5)]
    df_new['g_avg_result_fft'] = [i.mean() for i in ((pd.Series(gx_list_fft)**2 + pd.Series(gy_list_fft)**2 + pd.Series(gz_list_fft)**2)**0.5)]


    # signal magnitude area
    df_new['a_sma'] = pd.Series(ax_list).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(ay_list).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(az_list).apply(lambda x: np.sum(abs(x)/100))
    df_new['g_sma'] = pd.Series(gx_list).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(gy_list).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(gz_list).apply(lambda x: np.sum(abs(x)/100))

    df_new['a_sma_fft'] = pd.Series(ax_list_fft).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(ay_list_fft).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(az_list_fft).apply(lambda x: np.sum(abs(x)/100))
    df_new['g_sma_fft'] = pd.Series(gx_list_fft).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(gy_list_fft).apply(lambda x: np.sum(abs(x)/100)) \
                    + pd.Series(gz_list_fft).apply(lambda x: np.sum(abs(x)/100))

    return df_new

In [45]:
scaler = load('./weights/scaler.joblib')
model = load('./weights/SVC.joblib')

# For Dummy Data Testing

In [46]:
def generate_fake_data():
    df = pd.DataFrame([[np.random.uniform() for i in range(6)]], columns = ['ax', 'ay', 'az', 'gx', 'gy', 'gz'])
    return df

In [1]:
df = generate_fake_data()
i = 0
while i < 100:
    i += 1
    if len(df) < 50:
        df = df.append(generate_fake_data()).reset_index(drop=True)
    else:
        df = df.append(generate_fake_data()).iloc[1:].reset_index(drop=True)
        data = preprocess(df)
        X = scaler.transform(data.values)
        target = int(model.predict(X)[0])
        print("target:", target)